In [3]:
import csv
import json
import pandas as pd
import os
import glob
import datetime

# pow24 (hrPow, logHr) 폴더
# .json -> .csv

# search files in dir
plant_number    = '126' #126 순천, 140, 172 ? {
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "-------------------------------------------\n",
      "\t  > FILE DATE ERROR\n",
      "missing : 2019-09-12\t,existing file : 2019-09-13\t,interval : 1 day, 0:00:00\n",
      "missing : 2019-11-22\t,existing file : 2019-11-23\t,interval : 1 day, 0:00:00\n",
      "missing : 2019-11-30\t,existing file : 2019-12-01\t,interval : 1 day, 0:00:00\n",
      "missing : 2019-12-17\t,existing file : 2019-12-18\t,interval : 1 day, 0:00:00\n",
      "missing : 2020-05-01\t,existing file : 2020-05-03\t,interval : 2 days, 0:00:00\n",
      "-------------------------------------------\n",
      "\t  > FILE SIZE ERROR\n",
      "C:/solar/pow_24/UR00000126\\20191028.json/602\t22\n",
      "C:/solar/pow_24/UR00000126\\20191107.json/0\n",
      "C:/solar/pow_24/UR00000126\\20191108.json/0\n",
      "C:/solar/pow_24/UR00000126\\20191109.json/0\n",
      "C:/solar/pow_24/UR00000126\\20191110.json/0\n",
      "C:/solar/pow_24/UR00000126\\20191111.json/0\n",
      "C:/solar/pow_24/UR00000126\\20191112.json/0\n",
      "C:/solar/pow_24/UR00000126\\20200214.json/524\t19\n",
      "C:/solar/pow_24/UR00000126\\20200307.json/0\n",
      "C:/solar/pow_24/UR00000126\\20200308.json/0\n",
      "C:/solar/pow_24/UR00000126\\20200309.json/0\n",
      "C:/solar/pow_24/UR00000126\\20200310.json/0\n",
      "C:/solar/pow_24/UR00000126\\20200328.json/393\t14\n",
      "C:/solar/pow_24/UR00000126\\20200329.json/405\t14\n",
      "C:/solar/pow_24/UR00000126\\20200625.json/607\t22\n",
      "C:/solar/pow_24/UR00000126\\20200809.json/611\t22\n"
     ]
    }
   ],
   "source": [
    "import csv\n",
    "import json\n",
    "import pandas as pd\n",
    "import os\n",
    "import glob\n",
    "import datetime\n",
    "\n",
    "# pow24 (hrPow, logHr) 폴더\n",
    "# .json -> .csv\n",
    "\n",
    "# search files in dir\n",
    "plant_number    = '126' #126 순천, 140, 172 ? \n",
    "json_dir_name   = \"C:/solar/pow_24/UR00000\"+plant_number+\"/\"\n",
    "csv_dir_name    = \"C:/Users/VISLAB_PHY/Desktop/Workspace/Data/pow_24/UR00000\"+plant_number\n",
    "json_pattern    = os.path.join(json_dir_name,'*.json')\n",
    "file_list       = glob.glob(json_pattern)\n",
    "\n",
    "# err check variable\n",
    "err_file_size = []\n",
    "err_file_date = []\n",
    "start_date =\"2019-08-20\" \n",
    "file_date = \"0000-00-00\"\n",
    "base_date = datetime.datetime.strptime(start_date,\"%Y-%m-%d\").date()\n",
    "\n",
    "\n",
    "for file in file_list:\n",
    "    with open(file) as json_file:\n",
    "        \n",
    "        # 1. file date check\n",
    "        file_date = datetime.datetime.strptime(file[-13:-5], \"%Y%m%d\").date()\n",
    "        date_interval = file_date - base_date\n",
    "        if file_date != base_date:#err1\n",
    "            err_file_date.append('missing : '+str(base_date)+\"\\t,existing file : \"+str(file_date)+\"\\t,interval : \"+str(date_interval))\n",
    "        base_date = file_date + datetime.timedelta(days=1)\n",
    "\n",
    "        # 2. empty file check\n",
    "        file_size = os.path.getsize(file)\n",
    "        if file_size == 0:\n",
    "            err_file_size.append(file + \"/\" + str(file_size))#err2\n",
    "            continue\n",
    "    \n",
    "        # parsing json file\n",
    "        json_data = json.load(json_file)\n",
    "        data = json_data['result']\n",
    "\n",
    "        data_hrPow = []\n",
    "        data_logHr = []\n",
    "        \n",
    "        # missing data file check\n",
    "        # 단, 00시 데이터가 없을경우, 수기로 0채워줌\n",
    "        #  나머지 경우는, 결측값으로 인정\n",
    "        if len(data) < 22:\n",
    "            err_file_size.append(file + \"/\" + str(file_size) + '\\t' + str(len(data)))\n",
    "            continue\n",
    "    \n",
    "        if len(data) == 22:\n",
    "            if '00' != data[0]['logHr'] :\n",
    "                data_hrPow.append(0)\n",
    "                data_logHr.append('00')\n",
    "            else:\n",
    "                err_file_size.append(file + \"/\" + str(file_size) + '\\t' + str(len(data)))\n",
    "           \n",
    "        for line in data:\n",
    "            data_hrPow.append(line['hrPow'])\n",
    "            data_logHr.append(line['logHr'])\n",
    "                \n",
    "        df = pd.DataFrame(columns=['hrPow','logHr'])\n",
    "        df['hrPow'] = data_hrPow\n",
    "        df['logHr'] = data_logHr\n",
    "        \n",
    "        # save csv file\n",
    "        df.to_csv(csv_dir_name + '_csv/' + file[-13:-5] + '.csv', index=False, header=True)\n",
    "        \n",
    "# print err info.    \n",
    "print(\"-------------------------------------------\")\n",
    "print(\"\\t  > FILE DATE ERROR\")\n",
    "for f in err_file_date:\n",
    "    print(f)\n",
    "print(\"-------------------------------------------\")\n",
    "print(\"\\t  > FILE SIZE ERROR\")\n",
    "for f in err_file_size:\n",
    "    print(f)\n",
    "\n",
    "\n",
    "\n",
    "\n"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

json_dir_name   = "C:/solar/pow_24/UR00000"+plant_number+"/"
csv_dir_name    = "C:/Users/VISLAB_PHY/Desktop/Workspace/Data/pow_24/UR00000"+plant_number
json_pattern    = os.path.join(json_dir_name,'*.json')
file_list       = glob.glob(json_pattern)

# err check variable
err_file_size = []
err_file_date = []
start_date ="2019-08-20" 
file_date = "0000-00-00"
base_date = datetime.datetime.strptime(start_date,"%Y-%m-%d").date()


for file in file_list:
    with open(file) as json_file:
        
        # 1. file date check
        file_date = datetime.datetime.strptime(file[-13:-5], "%Y%m%d").date()
        date_interval = file_date - base_date
        if file_date != base_date:#err1
            err_file_date.append('missing : '+str(base_date)+"\t,existing file : "+str(file_date)+"\t,interval : "+str(date_interval))
        base_date = file_date + datetime.timedelta(days=1)

        # 2. empty file check
        file_size = os.path.getsize(file)
        if file_size == 0:
            err_file_size.append(file + "/" + str(file_size))#err2
            continue
    
        # parsing json file
        json_data = json.load(json_file)
        data = json_data['result']

        data_hrPow = []
        data_logHr = []
        
        # missing data file check
        # 단, 00시 데이터가 없을경우, 수기로 0채워줌
        #  나머지 경우는, 결측값으로 인정
        if len(data) < 22:
            err_file_size.append(file + "/" + str(file_size) + '\t' + str(len(data)))
            continue
    
        if len(data) == 22:
            if '00' != data[0]['logHr'] :
                data_hrPow.append(0)
                data_logHr.append('00')
            else:
                err_file_size.append(file + "/" + str(file_size) + '\t' + str(len(data)))
           
        for line in data:
            data_hrPow.append(line['hrPow'])
            data_logHr.append(line['logHr'])
                
        df = pd.DataFrame(columns=['hrPow','logHr'])
        df['hrPow'] = data_hrPow
        df['logHr'] = data_logHr
        
        # save csv file
        df.to_csv(csv_dir_name + '_csv/' + file[-13:-5] + '.csv', index=False, header=True)
        
# print err info.    
print("-------------------------------------------")
print("\t  > FILE DATE ERROR")
for f in err_file_date:
    print(f)
print("-------------------------------------------")
print("\t  > FILE SIZE ERROR")
for f in err_file_size:
    print(f)






-------------------------------------------
	  > FILE DATE ERROR
missing : 2019-09-12	,existing file : 2019-09-13	,interval : 1 day, 0:00:00
missing : 2019-11-22	,existing file : 2019-11-23	,interval : 1 day, 0:00:00
missing : 2019-11-30	,existing file : 2019-12-01	,interval : 1 day, 0:00:00
missing : 2019-12-17	,existing file : 2019-12-18	,interval : 1 day, 0:00:00
missing : 2020-05-01	,existing file : 2020-05-03	,interval : 2 days, 0:00:00
-------------------------------------------
	  > FILE SIZE ERROR
C:/solar/pow_24/UR00000126\20191028.json/602	22
C:/solar/pow_24/UR00000126\20191107.json/0
C:/solar/pow_24/UR00000126\20191108.json/0
C:/solar/pow_24/UR00000126\20191109.json/0
C:/solar/pow_24/UR00000126\20191110.json/0
C:/solar/pow_24/UR00000126\20191111.json/0
C:/solar/pow_24/UR00000126\20191112.json/0
C:/solar/pow_24/UR00000126\20200214.json/524	19
C:/solar/pow_24/UR00000126\20200307.json/0
C:/solar/pow_24/UR00000126\20200308.json/0
C:/solar/pow_24/UR00000126\20200309.json/0
C:/sol